In [1]:
#Import the right libraries 
import pandas as pd 
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pathlib
from distutils.dir_util import copy_tree
import os
import random
import shutil
import array as arr

In [2]:
import torch
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models
import tensorflow as tf

##Getting images just for train for now. Will get the other in later. But use the train first to test some type of model to see baseline errors 
BATCH_SIZE =40
IMG_HEIGHT = 200
IMG_WIDTH = 200

sets_dir = '~/test_instance/Sets'

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor()
    ]),
    'test': transforms.Compose([
        transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
        transforms.ToTensor()
    ])
}
directories = {'train':'Training', 'val':'Dev', 'test': 'Test'}
image_datasets = {x: datasets.ImageFolder(os.path.join(sets_dir, directories[x]),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=BATCH_SIZE,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Class Names: " + str(class_names))

Class Names: ['COVID', 'Non-COVID']


In [3]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            model.zero_grad()

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [4]:

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
            
        model.train(mode=was_training)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

model_ft = models.resnet18(pretrained=False)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
import time
import copy
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [5]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

model_dn121 = models.densenet121(pretrained=False)

model_dn121 = model_dn121.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_dn121 = optim.Adam(model_dn121.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_dn121, step_size=7, gamma=0.1)

In [6]:
import time
import copy
model_dn121 = train_model(model_dn121, criterion, optimizer_dn121, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 1.4954 Acc: 0.7743
val Loss: 0.9990 Acc: 0.5154

Epoch 1/24
----------
train Loss: 0.3446 Acc: 0.8655
val Loss: 1.2697 Acc: 0.6077

Epoch 2/24
----------
train Loss: 0.3273 Acc: 0.8665
val Loss: 1.1680 Acc: 0.5923

Epoch 3/24
----------
train Loss: 0.3094 Acc: 0.8809
val Loss: 0.8255 Acc: 0.5923

Epoch 4/24
----------
train Loss: 0.3160 Acc: 0.8770
val Loss: 0.7184 Acc: 0.7077

Epoch 5/24
----------
train Loss: 0.2799 Acc: 0.8809
val Loss: 0.6615 Acc: 0.7308

Epoch 6/24
----------
train Loss: 0.3042 Acc: 0.8713
val Loss: 0.7226 Acc: 0.7077

Epoch 7/24
----------
train Loss: 0.2428 Acc: 0.9030
val Loss: 0.8985 Acc: 0.6462

Epoch 8/24
----------
train Loss: 0.2235 Acc: 0.9078
val Loss: 0.9132 Acc: 0.6385

Epoch 9/24
----------
train Loss: 0.1995 Acc: 0.9289
val Loss: 0.6538 Acc: 0.7077

Epoch 10/24
----------
train Loss: 0.1853 Acc: 0.9289
val Loss: 0.7213 Acc: 0.7077

Epoch 11/24
----------
train Loss: 0.1773 Acc: 0.9289
val Loss: 0.6479 Acc: 0.7231

Ep

In [ ]:
!pip install pytorch-ignite

In [ ]:

import ignite
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
import ignite.metrics as mets

def thresholded_output_transform(output):
    y_pred, y = output
    y_pred = torch.round(y_pred)
    return y_pred, y

trainer = create_supervised_trainer(model_dn121, optimizer_dn121, criterion, device = device)
evaluator = create_supervised_evaluator(model_ft, metrics={'accuracy': mets.Accuracy(),'nll': mets.Loss(criterion), 'precision': mets.Precision(output_transform=thresholded_output_transform,average=False), 'recall':mets.Recall(output_transform=thresholded_output_transform,average=False)}, device=device)

In [ ]:
@trainer.on(Events.ITERATION_COMPLETED(every=100))
def log_training(trainer):
    batch_loss = trainer.state.output
    lr = optimizer.param_groups[0]['lr']
    e = trainer.state.epoch
    n = trainer.state.max_epochs
    i = trainer.state.iteration
    print("Epoch {}/{} : {} - batch loss: {}, lr: {}".format(e, n, i, batch_loss, lr))

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(dataloaders['train'])
    metrics = evaluator.state.metrics
    print("Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}, Precision: {}, Recall: {} "
          .format(trainer.state.epoch, metrics['accuracy'], metrics['nll'], metrics['precision'][0:1], metrics['recall'][0:1]))

@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(trainer):
    evaluator.run(dataloaders['val'])
    metrics = evaluator.state.metrics
    print("Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}, Precision: {}, Recall: {}"
          .format(trainer.state.epoch, metrics['accuracy'], metrics['nll'], metrics['precision'][0:1], metrics['recall'][0:1]))

trainer.run(dataloaders['train'], max_epochs=20)